# Qiitaの本文からタイトルを生成するためのデータセットを作る

- Qiitaの本文からタイトルを生成する記事: [【人工知能】深層学習で「記事タイトルを自動生成」する](https://qiita.com/sonoisa/items/30876467ad5a8a81821f)

※記事で用いたデータセットと類似のものを作るためのコードです。必ずしも同じものができる訳ではありません。

## Qiita APIのアクセストークン設定

※[自身のアカウントのアプリケーション設定](https://qiita.com/settings/applications)にて発行した個人用アクセストークン（スコープはread_qiitaのみ）を次の変数<code>QIITA_API_TOKEN</code>に書いてから、以降を実行してください。

※アクセストークンはパスワードと同等のものです。他人には絶対に教えていけません。

In [10]:
QIITA_API_TOKEN = ""  # 個人用アクセストークン

## [Qiitaの殿堂](http://youwht.ga)からLGTMの多い記事の一覧を取得する

In [1]:
!mkdir -p /content/dendou

In [2]:
year_month_list = [f"{year}{month:02}" 
                   for year in range(2012, 2020) 
                   for month in range(1, 13)]

In [3]:
import time
from tqdm import tqdm

for year_month in tqdm(year_month_list):
    !wget -qO /content/dendou/{year_month}_DEN.csv http://youwht.ga/static/DENDOU/{year_month}_DEN.csv
    time.sleep(1)

100%|██████████| 96/96 [02:08<00:00,  1.33s/it]


In [4]:
import glob
import csv
import xml.etree.ElementTree as ET
import re
from tqdm import tqdm

LINK_PATTERN = re.compile(r'<a href="([^"]+)" target="_blank" title="([^"]+)">(.*?)(</a>)?')

data_files = sorted(glob.glob("/content/dendou/*.csv"))
dataset = []

items = set()

for data_file in tqdm(data_files):
    with open(data_file, "r", encoding="utf-8") as f_in:
        reader = csv.reader(f_in)
        header = next(reader)  # skip header

        for row in reader:
            lgtm = row[1]
            link_match = LINK_PATTERN.match(row[2])
            if link_match is None:
                # print(row[2])
                continue
            
            href = link_match[1]
            tags = link_match[2]
            title = link_match[3]
            
            if href not in items:
                items.add(href)
                dataset.append({
                    "title": title,
                    "url": href,
                    "lgtm": int(lgtm),
                    "tags": tags
                })


100%|██████████| 96/96 [00:00<00:00, 1025.12it/s]


In [5]:
print(f"総記事数: {len(dataset)}")

総記事数: 9600


In [6]:
lgtms = [data["lgtm"] for data in dataset if data["lgtm"] >= 50]

In [7]:
print(f"LGTMが50以上の記事数: {len(lgtms)}")
print(f"LGTMの最小値: {min(lgtms)}")
print(f"LGTMが最大値: {max(lgtms)}")

LGTMが50以上の記事数: 8862
LGTMの最小値: 50
LGTMが最大値: 9891


## 記事の本文をQiita APIで取得する

本文の取得には12時間程度かかります。  
Qiita APIは1時間に1000回までという制限があることに注意。

In [8]:
import requests
import json

def extract_item_id(url):
    return re.sub("https?://qiita.com/[^/]+/items/", "", url)

def get_content(item_id):
    headers = {
        "Authorization": f"Bearer {QIITA_API_TOKEN}"
    }
    response = requests.get(f"https://qiita.com/api/v2/items/{item_id}", headers=headers)
    if response.status_code == 200:
        jsonDoc = response.json()
        return jsonDoc["body"], jsonDoc["title"], jsonDoc["likes_count"], jsonDoc["rendered_body"]
    else:
        print(f"WARN: {response.status_code}, {item_id}")
        return None, None, None, None

In [ ]:
import time
from tqdm import tqdm

for data in tqdm(dataset):
    if "body" in data:
        continue

    item_id = extract_item_id(data["url"])
    data["item_id"] = item_id
    body, title, likes_count, rendered_body = get_content(item_id)
    
    if body is not None:
        data["body"] = body
        data["title"] = title
        data["lgtm"] = likes_count
        data["rendered_body"] = rendered_body
    
    time.sleep(3.7)  # Qiita APIの呼び出しは1時間に1000回まで

In [ ]:
import json

with open("/content/qiita_hit_items.json", "w", encoding="utf-8") as f_out:
    json.dump(dataset, f_out, ensure_ascii=False)